# Benchmarking zarr using dask
The first thing we need to do is convert the netcdf files to dask data-frames that files.

In [11]:
import matplotlib.pyplot as plt
import os
import time
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
from Script.utils import printRed, printGreen, printHeader

In [12]:
cluster = LocalCluster(n_workers=32)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 32,Total memory: 503.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36063,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 503.49 GiB
Comm: tcp://127.0.0.1:39695,Total threads: 1
Dashboard: http://127.0.0.1:45705/status,Memory: 15.73 GiB
Nanny: tcp://127.0.0.1:37101,


In [ ]:
#shutdown
client.close()

In [13]:
path = "/anvil/datasets/ncar/AORC_Forcing/2016/"
files = os.listdir(path)[:10]
dataset = xr.open_mfdataset(
  [os.path.join(path, file) for file in files], 
  parallel=True,
  combine='nested',
  concat_dim='time',
  engine = "netcdf4")

In [14]:
dataset["RAINRATE"]

<xarray.DataArray 'RAINRATE' (time: 10, y: 3840, x: 4608)>
dask.array<concatenate, shape=(10, 3840, 4608), dtype=float32, chunksize=(1, 768, 922), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
  * time     (time) datetime64[ns] 2016-11-28T05:00:00 ... 2016-07-24T01:00:00
Attributes:
    cell_methods:    time: mean
    remap:           remapped via ESMF regrid_with_weights: Bilinear
    grid_mapping:    crs
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    proj4:           +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0...
    units:           mm s^-1
    standard_name:   precipitation_flux
    long_name:       Surface Precipitation Rate

In [22]:
dataset = dataset.chunk({"time": 7, "x": 860, "y": 100})

In [21]:
dataset['RAINRATE']

<xarray.DataArray 'RAINRATE' (time: 10, y: 3840, x: 4608)>
dask.array<rechunk-merge, shape=(10, 3840, 4608), dtype=float32, chunksize=(10, 100, 860), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
  * time     (time) datetime64[ns] 2016-11-28T05:00:00 ... 2016-07-24T01:00:00
Attributes:
    cell_methods:    time: mean
    remap:           remapped via ESMF regrid_with_weights: Bilinear
    grid_mapping:    crs
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    proj4:           +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0...
    units:           mm s^-1
    standard_name:   precipitation_flux
    long_name:       Surface Precipitation Rate

In [25]:
dataset.to_zarr("/anvil/scratch/x-michaelade/test.zarr", mode="w")

Next, we can try various chunking schemes. We will try varying temporal and spatial dimensions. We will always use squares for spatial dimensions as we cannot assume that latitude would be more beneficial than longituude in use cases. First, let us test with chunks across time.

Lets now define a function that can take the files, load them in zarr, and run some testcases on them

In [26]:
ds = xr.open_zarr("/anvil/scratch/x-michaelade/test.zarr")

In [28]:
ds['RAINRATE']

<xarray.DataArray 'RAINRATE' (time: 10, y: 3840, x: 4608)>
dask.array<open_dataset-RAINRATE, shape=(10, 3840, 4608), dtype=float32, chunksize=(7, 100, 860), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2016-11-28T05:00:00 ... 2016-07-24T01:00:00
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    cell_methods:    time: mean
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       Surface Precipitation Rate
    proj4:           +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0...
    remap:           remapped via ESMF regrid_with_weights: Bilinear
    standard_name:   precipitation_flux
    units:           mm s^-1

In [1]:
import s3fs
import xarray as xr

url = "s3://noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr"
fs = s3fs.S3FileSystem(anon=True)
cloud_ds = xr.open_zarr(s3fs.S3Map(url, s3=fs))

In [2]:
cloud_ds['RAINRATE']

<xarray.DataArray 'RAINRATE' (time: 385704, y: 3840, x: 4608)>
dask.array<open_dataset-RAINRATE, shape=(385704, 3840, 4608), dtype=float32, chunksize=(672, 350, 350), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01 ... 2023-01-31T23:00:00
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    cell_methods:    time: mean
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       Surface Precipitation Rate
    proj4:           +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0...
    remap:           remapped via ESMF regrid_with_weights: Bilinear
    standard_name:   precipitation_flux
    units:           mm s^-1

In [8]:
times = [
  [0, 1],
  [0, 1],
  [0, 1],
  [0, 1],
  [0, 1]
         ]

In [12]:
import numpy as np
times = np.array(times)

print(times[:, 1])

[1 1 1 1 1]
